# Google Drive Python API Tutorial

The purpose of this notebook is to demonstrate the different capabilities of the Google Drive Python API. Note that the reader is encouraged to first check if [PyDrive](https://pythonhosted.org/PyDrive/) can do the job. PyDrive is mainly used for simple requests, such as creating folders and files. However, in the case that a more complex task is required, then the Google Drive Python API is more appropriate. This guide may be complemented with the official [documentation](http://googleapis.github.io/google-api-python-client/docs/dyn/drive_v3.html).

A very helpful quickstart guide can be found [here](https://www.thepythoncode.com/article/using-google-drive--api-in-python).

Note that the Google Drive API is a REST API. This means that its overall architecture is based on a known blueprint. This blueprint is detailed [here](https://restfulapi.net/).

To get started, complete step 1 and step 2 on this [page](https://developers.google.com/drive/api/v3/quickstart/python) to download a `credentials.json` file into your working directory and to install the Google Python client library.

Once completed, the next step is gain access to your Google Drive using the API. First, define what kind of read and write permissions you will have:

In [ ]:
SCOPES = ['https://www.googleapis.com/auth/drive.file']

In this case, you will have the ability to create and upload files using the Google Drive API. However, other types of permissions are available. See [here](https://developers.google.com/drive/api/v3/about-auth) for details.

Next, check if the user's access and refresh tokens already exist:

In [ ]:
import pickle
import os

if os.path.exists('token.pickle'):
    with open('token.pickle','rb') as f:
        creds = pickle.load(f)
else:
    creds = None

If the user's access and refresh tokens don't exist, or they are no longer valid, then create them and save them. This will **not** work if the `credentials.json` file is not in your working directory.

In [ ]:
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    
    with open('token.pickle', 'wb') as f:
        pickle.dump(creds,f)

Now check that the authentication flow worked:

In [ ]:
creds.client_id

This should be the same as the string in the `"cliend_id"` field in the `credentials.json` file. Additionally, if the access and refresh tokens did not already exist, then a `token.pickle` file should now be in your working directory.

Finally, a `drive` object is created to directly interface with your Google Drive:

In [ ]:
from googleapiclient.discovery import build

drive = build('drive', 'v3',credentials=creds)

The `drive` object has several associated public methods, including `files`, `drives`, and `comments`:

In [ ]:
dir(drive)

These public methods have different purposes. Details on their purposes can be found [here](https://developers.google.com/drive/api/v3/reference).

For the purpose of this tutorial, however, only the `files` method will be used. The associated public methods of `drive.files()` are detailed [here](https://developers.google.com/drive/api/v3/reference/files).

In [ ]:
dir(drive.files())

## Create a folder

In the Google Drive API, files and folders are treated almost the same way. Both require the definition of their metadata before creation. More information about the metadata for a file/folder can be found [here](https://developers.google.com/drive/api/v3/reference/files#resource-representations). This metadata is defined using a Python dictionary.

For example, to create a folder, its metadata could be:

In [ ]:
folder_metadata = {
    
    'name': 'TestFolder',
    'mimeType': 'application/vnd.google-apps.folder'
    
}

The `'mimeType'` key refers to the type of media (file, folder, text file, PDF,...) to be created. More information about this can be found [here](https://stackoverflow.com/questions/3828352/what-is-a-mime-type). A list of available MIME types that are identifiable by the Google Drive API can be found [here](https://developers.google.com/drive/api/v3/mime-types).

Note that that the `body` parameter is available to all `drive.files()` methods that have a request body, including `create()` and `update()`. Here is an [example](https://developers.google.com/drive/api/v3/reference/files/update#request-body).

Next, a folder is created using the `create()` method:

In [ ]:
create_folder_req = drive.files().create(body = folder_metadata,fields = 'id,name')

The `body` parameter takes the metadata dictionary as input. The `fields` parameter is used to choose specific metadata to be returned in the response body of the HTTP request. More details on the format of the `fields` parameter can be found [here](https://developers.google.com/drive/api/v3/fields-parameter#formatting_rules_for_the_fields_parameter). More information about the available parameters for the `create()` method can be found [here](http://googleapis.github.io/google-api-python-client/docs/dyn/drive_v3.files.html#create).

More precisely, since `create_folder_req` is a HTTP request:

In [ ]:
create_folder_req

Then it has an associated response body. Also, since `create_folder_req` is a HTTP request, then it needs to be executed:

In [ ]:
folder = create_folder_req.execute()

The associated metadata fields that were requested in `create_folder_req` are then returned in the response body:

In [ ]:
folder

Every file and folder in Google Drive has a unique `id`. This means that two files or folders can have the same `name` with a different `id`.

## Upload a file

Since the `TestFolder` folder has been created, a file can be uploaded to it. First, create a sample `HelloWorld.txt` file to upload:

In [ ]:
with open('HelloWorld.txt','w') as f:
    f.write('Hello World!')

Next, create metadata for this file:

In [ ]:
file_metadata = {
    'name': 'HelloWorld.txt',
    'parents': [folder['id']]
}

The `'parents'` field indicates the folders that should contain this file. Note that in Google Drive, several folders can contain the same file. In this case, the `id` of the `TestFolder` folder is specified as the parent folder.

Next, a `MediaFileUpload` object is created to complete the upload:

In [ ]:
from googleapiclient.http import MediaFileUpload

media = MediaFileUpload('HelloWorld.txt',chunksize = -1,resumable = False)

The `chunksize` parameter indicates the size of each piece of the file to be consecutively uploaded. A `chunksize` of `-1` indicates that the whole file should be uploaded as one piece, which is only possible if the file is less than 5 MB in size. The `resumable` parameter indicates whether the upload should be done over multiple requests or not. In this case, the upload is done using a single request. More details on the parameters for the `MediaFileUpload` object can be found [here](https://github.com/googleapis/google-api-python-client/blob/21af37b11ea2d6a89b3df484e1b2fa1d12849510/googleapiclient/http.py#L534).

Finally, upload the file:

In [ ]:
file = drive.files().create(body = file_metadata,media_body = media,fields = 'id').execute()

The `id` of the uploaded `HelloWorld.txt` file is:

In [ ]:
file['id']

## Delete a file

The uploaded `HelloWorld.txt` file can be deleted using the `delete()` method:

In [ ]:
drive.files().delete(fileId = file['id']).execute()

Note that if the `fileId` is the `id` of a folder, then the folder and the files within it are deleted. See [here](https://developers.google.com/drive/api/v3/reference/files/delete) for details.

## Make a file indexable

There are three ways to make a file indexable (searchable):

1. Using the `contentHints.indexableText` metadata field.
2. Using the `description` metadata field.
3. Using the `properties` metadata field.

### `contentHints.indexableText` metadata field

For example, suppose that the uploaded `HelloWorld.txt` file was associated with a specific day of the week, such as Monday. Then its metadata can be:

In [ ]:
file_metadata = {
    'name': 'HelloWorld.txt',
    'parents': [folder['id']],
    'contentHints': {'indexableText':'day of week: Monday'}
}

The `HelloWorld.txt` file can then be re-uploaded with the new metadata:

In [ ]:
media = MediaFileUpload('HelloWorld.txt',chunksize = -1,resumable = False)
file = drive.files().create(body = file_metadata,media_body = media,fields = '*').execute()

The sentence `day of week: Monday` can then be entered into the search bar in Google Drive to return this file.

Notice that the `contentHints` field does not show up in the response body:

In [ ]:
file

Also, there is an inherent problem with this approach. The `contentHints.indexableText` field is included in the body of text that is searched by Google Drive when searching file contents. This means that if there are PDF, text, or any other content-indexable files in the drive, and any of them happen to contain `day of week: Monday`, then they will also be returned with `HelloWorld.txt` when performing a search.

### `description` metadata field

Alternatively, the `description` metadata field can be used. However, instead of deleting the original `HelloWorld.txt` file then re-uploading it with new metadata, its metadata can be updated while it is in the drive using the `update()` method:

In [ ]:
new_file_metadata = {
    'contentHints': '', # remove previous metadata
    'description': 'day of week: Monday'
}

file = drive.files().update(fileId = file['id'],body = new_file_metadata,fields = '*').execute()

Notice that the `description` field is visible in the response body:

In [ ]:
file

The only advantage that the `description` field offers over the `contentHints.indexableText` field is that it is visible when right-clicking a file and selecting the 'View details' option. However, it suffers from the same problem, since searching for `day of week: Monday` in the search bar also yields other files that contain this sentence. This is because it is not possible to restrict the search to only use the `description` field. See [here](https://support.google.com/drive/answer/2375114?hl=en&authuser=1) for the types of filters that can be used while searching using the Google Drive search bar.

### `properties` metadata field

The solution to this problem is to use the `properties` metadata field instead, which can be searched using the Google Drive API. See [here](https://developers.google.com/drive/api/v3/properties) and [here](https://developers.google.com/drive/api/v3/search-files) for details.

In [ ]:
properties = {'day_of_week' : 'Monday'}

new_file_metadata = {
    'description': '', # remove previouse metadata
    'properties':properties
}

file1 = drive.files().update(fileId = file['id'],
                             body = new_file_metadata,
                             fields = 'name,id,properties').execute()

Note that the keys in the `properties` dict cannot contain spaces. The `properties` field has a standard format that must be followed, which is `properties = {'key1' : 'value1', 'key2' : 'value2', ...}`. Here is the response body for the file:

In [ ]:
file1

Next, the `files().list()` method can be used to search for the `HelloWorld.txt` file. Note that in this case, the parameters in the `fields` parameter need to be enclosed with `files()`.

In [ ]:
query = "properties has {key = 'day_of_week' and value = 'Monday'}"

drive.files().list(q = query,fields = 'files(name,properties)').execute()

### Search using multiple properties

Additionally, multiple properties can be defined to filter search results. To demonstrate this, the properties of the original `HelloWorld.txt` file will be updated and two other files `HelloWorld2.txt` and `HelloWorld3.txt` with different properties will be uploaded:

In [ ]:
properties = {
    'day_of_week' : 'Monday',
    'month' : 'June',
    'color' : 'blue'
}

new_file_metadata = {'properties':properties}

file1 = drive.files().update(fileId = file['id'],
                             body = new_file_metadata,
                             fields = 'name,id,properties').execute()

file1

Next, create and upload `HelloWorld2.txt` and `HelloWorld3.txt` with different parameters:

In [ ]:
with open('HelloWorld2.txt','w') as f:
    f.write('Hello World!')

properties = {
    'day_of_week' : 'Monday',
    'month' : 'August',
    'color' : 'green'
}

file_metadata = {
    'name': 'HelloWorld2.txt',
    'parents': [folder['id']],
    'properties': properties
}

media = MediaFileUpload('HelloWorld2.txt',chunksize = -1,resumable = False)
file2 = drive.files().create(body = file_metadata,media_body = media,fields = 'id,name,properties').execute()

file2

In [ ]:
with open('HelloWorld3.txt','w') as f:
    f.write('Hello World!')

properties = {
    'day_of_week' : 'Tuesday',
    'month' : 'September',
    'color' : 'blue'
}

file_metadata = {
    'name': 'HelloWorld3.txt',
    'parents': [folder['id']],
    'properties': properties
}

media = MediaFileUpload('HelloWorld3.txt',chunksize = -1,resumable = False)
file3 = drive.files().create(body = file_metadata,media_body = media,fields = 'id,name,properties').execute()

file3

Finally, `files().list()` can be used to search for these files. First, to return `HelloWorld.txt` and `HelloWorld2.txt`:

In [ ]:
query = "properties has {key = 'day_of_week' and value = 'Monday'}"

drive.files().list(q = query,fields = 'files(name,properties)').execute()

Next, to return `HelloWorld2.txt` only:

In [ ]:
query = "properties has {key = 'day_of_week' and value = 'Monday'} and properties has {key = 'color' and value = 'blue'}"

drive.files().list(q = query,fields = 'files(name,properties)').execute()

Notice that the general format to query multiple properties at the same time is:

`"properties has {key = 'key1' and value = 'value1'} and properties has {key = 'key2' and value = 'value2'} and properties has {key = 'key3' and value = 'value3'} and ..."`

As another example, to return `HelloWorld3.txt` only:

In [ ]:
query = "properties has {key = 'month' and value = 'September'} and properties has {key = 'day_of_week' and value = 'Tuesday'}"

drive.files().list(q = query,fields = 'files(name,properties)').execute()